# ⛳ **Playlist는 내가 만들게, 제목은 누가 입력할래?**

# 🧰 **Utils**

In [ ]:
PYTHONHASHSEED = 0

In [ ]:
import warnings

warnings.filterwarnings(action='ignore') 

In [ ]:
import os,io
import json
import time
import pickle

from collections import Counter
from itertools import chain

from tqdm import tqdm
import numpy as np

from gensim.models import Word2Vec
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors

dir = '/content/drive/MyDrive/Melon-PL-Continuation/11월 수정중   카카오 아레나 (멜론 추천)/1206 word2vec'

In [ ]:
def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

def load_json(fname):
    with open(fname, encoding="utf-8") as f:
        json_obj = json.load(f)

    return json_obj

In [ ]:
class ArenaEvaluator:
    def _idcg(self, l):
        return sum((1.0 / np.log(i + 2) for i in range(l)))

    def __init__(self):
        self._idcgs = [self._idcg(i) for i in range(101)]

    def _ndcg(self, gt, rec):
        dcg = 0.0
        for i, r in enumerate(rec):
            if r in gt:
                dcg += 1.0 / np.log(i + 2)

        return dcg / self._idcgs[len(gt)]

    def _load_json(self,fname):
        with open(fname, encoding="utf-8") as f:
            json_obj = json.load(f)

        return json_obj

    def _eval(self, gt_fname, rec_fname):
        gt_playlists = self._load_json(gt_fname)
        gt_dict = {g["id"]: g for g in gt_playlists}
        rec_playlists = self._load_json(rec_fname)

        gt_ids = set([g["id"] for g in gt_playlists])
        rec_ids = set([r["id"] for r in rec_playlists])

        if gt_ids != rec_ids:
            raise Exception("결과의 플레이리스트 수가 올바르지 않습니다.")

        rec_song_counts = [len(p["songs"]) for p in rec_playlists]
        rec_tag_counts = [len(p["tags"]) for p in rec_playlists]

        if set(rec_song_counts) != set([100]):
            raise Exception("추천 곡 결과의 개수가 맞지 않습니다.")

        if set(rec_tag_counts) != set([10]):
            raise Exception("추천 태그 결과의 개수가 맞지 않습니다.")

        rec_unique_song_counts = [len(set(p["songs"])) for p in rec_playlists]
        rec_unique_tag_counts = [len(set(p["tags"])) for p in rec_playlists]

        if set(rec_unique_song_counts) != set([100]):
            raise Exception("한 플레이리스트에 중복된 곡 추천은 허용되지 않습니다.")

        if set(rec_unique_tag_counts) != set([10]):
            raise Exception("한 플레이리스트에 중복된 태그 추천은 허용되지 않습니다.")

        music_ndcg = 0.0
        tag_ndcg = 0.0

        for rec in rec_playlists:
            gt = gt_dict[rec["id"]]
            music_ndcg += self._ndcg(gt["songs"], rec["songs"][:100])
            tag_ndcg += self._ndcg(gt["tags"], rec["tags"][:10])

        music_ndcg = music_ndcg / len(rec_playlists)
        tag_ndcg = tag_ndcg / len(rec_playlists)
        score = music_ndcg * 0.85 + tag_ndcg * 0.15

        return music_ndcg, tag_ndcg, score

    def evaluate(self, gt_fname, rec_fname):
        try:
            music_ndcg, tag_ndcg, score = self._eval(gt_fname, rec_fname)
            print(f"Music nDCG: {music_ndcg:.6}")
            print(f"Tag nDCG: {tag_ndcg:.6}")
            print(f"Score: {score:.6}")
        except Exception as e:
            print(e)

# 🏃‍♀ **Data 준비하기**

In [ ]:
def get_data(test = True):
    if test:
        train_path = os.path.join(dir, 'train.json')
        val_path = os.path.join(dir, 'val_questions.json')
    else:
        pass

    base_path = os.path.join(dir, 'base_results_gep.json')

    train = load_json(train_path)
    val = load_json(val_path)
    base_res = load_json(base_path)

    return train, val, base_res

In [ ]:
train, val, base_res = get_data(test = True)

# 📻 **Song meta 가져오기**

In [ ]:
import pandas as pd

song_meta = pd.read_json('/content/drive/MyDrive/Melon-PL-Continuation/0802/train_split/song_meta.json', typ = 'frame')

# 🎶 **Playlist2Vec Build**

In [ ]:
class Playlist2Vec:
    def __init__(self,train, val, results):
        self.train = train
        self.val = val
        self.results = {}
        for ply in results:
            self.results[ply['id']] = {'songs':list(map(str,ply['songs'])),'tags':ply['tags']}
        self.data = self.train + self.val

        print('*** Build Vocab ***')
        self.build_vocab()

    def build_vocab(self):
        self.id_to_songs = {}
        self.id_to_tags = {}
        self.corpus = []

        for ply in self.data:
            self.id_to_songs[str(ply['id'])] = [*map(str,ply['songs'])]
            self.id_to_tags[str(ply['id'])] = [*map(str,ply['tags'])]

            items = self.id_to_songs[str(ply['id'])] + self.id_to_tags[str(ply['id'])]
            if len(items) > 1:
                self.corpus.append(items)

        self.songs = set(chain.from_iterable(self.id_to_songs.values())) 
        self.tags = set(chain.from_iterable(self.id_to_tags.values())) 

        print("> Corpus :", len(self.corpus))
        print(f'> Songs + Tags = {len(self.songs)} + {len(self.tags)} = {len(self.songs) + len(self.tags)}')
        print("> Playlist Id Type :", type(list(self.id_to_songs.keys())[0]),type(list(self.id_to_tags.keys())[0]))

    def register_w2v(self, w2v_model):
        self.w2v_model = w2v_model
        self.p2v_model = WordEmbeddingsKeyedVectors(self.w2v_model.trainables.layer1_size)

    def train_w2v(self, min_count = 3, size = 128, window = 210, negative = 5, sg = 1, hs = 0, workers = 1):
        # workers = 1 ; for consistency
        self.w2v_model = Word2Vec(sentences = self.corpus, min_count= min_count , size = size , window = window, negative = negative , sg = sg, hs = hs, workers = workers)
        self.p2v_model = WordEmbeddingsKeyedVectors(self.w2v_model.trainables.layer1_size)

    def build_p2v(self):
        start = time.time()
        pids = []
        playlist_embedding = []

        # for pid in tqdm(self.id_to_songs.keys()):
        for pid in self.id_to_songs.keys():
            if len(self.id_to_songs[pid]) < 1:
                continue

            ply_embedding = 0

            for item in self.id_to_songs[pid] + self.id_to_tags[pid]:
                if self.w2v_model.wv.vocab.get(str(item)) is None:
                    # train data에 없었거나, w2v 학습 과정에서 min_counts에 걸러진 경우
                    continue

                ply_embedding += self.w2v_model.wv.get_vector(str(item))

            if type(ply_embedding) != int: # 한 번이라도 update 되었다면
                pids.append(str(pid)) # ! string !
                playlist_embedding.append(ply_embedding)

        self.p2v_model.add(pids,playlist_embedding)

        print(f'> running time : {time.time()-start:.3f}')
        print(f'> Register (ply update) : {len(pids)} / {len(self.id_to_songs)}')
        val_ids = set([str(p["id"]) for p in self.val])
        print(f'> Only {len( val_ids - set(pids) )} of validation set ( total : {len(val_ids)} ) can not find similar playlist in train set.')

    def remove_seen(self,seen, recommend, attr):
        res = []
        limit = [100,10][attr == 'tags']

        for item in recommend:
            if item not in seen and item not in res:
                res.append(item)
                if len(res) == limit:
                    break
        return res

    def build_answers(self):
        self.answers = []
        use_popular = 0
        lack_info = Counter()

        # for ply in tqdm(self.val):
        for ply in self.val:
            if self.p2v_model.vocab.get(str(ply['id'])) is not None:

                ply_candidates = self.p2v_model.most_similar(str(ply['id']), topn = 200)
                song_candidates = []
                tag_candidates = []

                for cid , _ in ply_candidates:
                    song_candidates.extend(self.id_to_songs[str(cid)])
                    tag_candidates.extend(self.id_to_tags[str(cid)])

                song_most_common = [song for song,_ in Counter(song_candidates).most_common()]
                tag_most_common = [tag for tag,_ in Counter(tag_candidates).most_common()]

                if len(song_most_common) < 100:
                    lack_info[len(song_most_common)] += 1
            else:
                use_popular += 1
                song_most_common = []
                tag_most_common = []

            song_rec = self.remove_seen(ply['songs'], song_most_common + self.results[ply['id']]['songs'], 'songs')
            tag_rec = self.remove_seen(ply['tags'], tag_most_common + self.results[ply['id']]['tags'], 'tags')

            assert sum([type(s) != str for s in song_rec]) == 0
            assert len(set(song_rec)) == 100
            assert len(set(tag_rec)) == 10

            self.answers.append({
                'id' : ply['id'],
                'songs' : list(map(int,song_rec)),
                'tags' : tag_rec
            })

        print('> use_all_popular :', use_popular)
        print('> lack_info :', sum(lack_info.values()))
        print('            :',lack_info)

In [ ]:
model = Playlist2Vec(train, val, base_res)

*** Build Vocab ***
> Corpus : 112010
> Songs + Tags = 521583 + 23421 = 545004
> Playlist Id Type : <class 'str'> <class 'str'>


In [ ]:
# 학습후 저장해놓은 w2v 모델 등록

with open(os.path.join(dir,'w2v_128.pkl'), 'rb') as f:
    w2v_model = pickle.load(f)

model.register_w2v(w2v_model)

In [ ]:
model.build_p2v()

> running time : 12.246
> Register (ply update) : 91930 / 96639
> Only 4697 of validation set ( total : 23015 ) can not find similar playlist in train set.


# 🧐 **Song,Tag Embedding 학습 결과 살펴보기**
1. model.w2v_model.wv.**similar_by_word**('랩')
```python
[('힙합', 0.9615921378135681),
 ('HipHop', 0.7985354661941528),
 ('Rap', 0.7893173694610596),
 ('국내힙합', 0.7886797189712524)]
```
2. **most_similar** : sum(positive) - sum(negative)  
p = ['아빠','여성'] , n = ['남성'] => 결과 :['엄마']
```python
[('우울', 0.8385971188545227),
 ('외로움', 0.8382763862609863),
 ('쓸쓸함', 0.8005260229110718),
 ('외로운', 0.8002836108207703)]
```

3. model.w2v_model.wv.**similarity**('랩','힙합')  
  cosine 유사도 return

4. model.w2v_model.wv.**similar_by_vector**
```python
v1 = model.w2v_model.wv.get_vector('랩')
v2 = model.w2v_model.wv.get_vector('힙합')
model.w2v_model.wv.similar_by_vector(v1+v2)
```
```python
[('랩', 0.9919853806495667),
 ('힙합', 0.988567054271698),
 ('국내힙합', 0.8164964318275452),
 ('HipHop', 0.8128796815872192),
 ('Rap', 0.800581157207489),
 ('국힙', 0.7753417491912842)]
```

In [ ]:
tags = [k for k in model.w2v_model.wv.vocab.keys() if k.isalpha()]
print(len(tags))
print(tags[:30])

6746
['불쾌지수', '친구들과', '눈꽃', '추천합니다', '사랑스러운노래', '맥주펍', '피쳐링미침', 'CF음악', '월요병저리가', 'NAS', '록스피릿', '컬트무비', '힐림', '모아나', '나윤권', 'SHINEE', '두근거리는', '찰떡궁합', '부스터', '뮤지컬음악', '카페플레이리스트', 'ROMANTIC', '반려묘', '찰리XCX', '여름노래모음', '써머', '광주', '댄스힙합', '집에서즐기는', '슈크박스']


## **가장 가까운 태그 둘러보기**

In [ ]:
def top_items(query_list):
    candidates = model.w2v_model.wv.most_similar(query_list, topn = 1000)
    tags = [item for item,sim in candidates if not item.isdigit()][:20]
    songs = [item for item,sim in candidates if item.isdigit()][:10]

    print('> 유사 태그 :', tags)
    print('> 유사곡 :')
    display(song_meta.iloc[songs])

### **쓸쓸, 쓸쓸+쓸쓸한, 쓸쓸+쓸쓸한+쓸쓸함의 결과에는 큰 차이가 없다.**
즉 유사한 vector로 잘 embedding되었다.

In [ ]:
model.w2v_model.wv.most_similar(['쓸쓸'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('우울', 0.8182780742645264),
 ('외로움', 0.7527623772621155),
 ('눈물', 0.7503553032875061),
 ('헤어짐', 0.7471939921379089),
 ('쓸쓸한', 0.7417927384376526),
 ('쓸쓸함', 0.7417812347412109),
 ('센치', 0.7397032976150513),
 ('혼자', 0.7139500975608826),
 ('차분', 0.7002543210983276),
 ('그리움', 0.698684811592102)]

In [ ]:
model.w2v_model.wv.most_similar(['쓸쓸','쓸쓸한'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('우울', 0.8385971188545227),
 ('외로움', 0.8382763862609863),
 ('쓸쓸함', 0.8005260229110718),
 ('외로운', 0.8002836108207703),
 ('헤어짐', 0.7965164184570312),
 ('혼자', 0.7933205366134644),
 ('눈물', 0.7925050258636475),
 ('센치', 0.7924936413764954),
 ('그리움', 0.7756912708282471),
 ('슬픈', 0.7600284218788147)]

In [ ]:
model.w2v_model.wv.most_similar(['쓸쓸','쓸쓸한','쓸쓸함'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('외로움', 0.8746573328971863),
 ('우울', 0.8421176671981812),
 ('헤어짐', 0.8126764297485352),
 ('눈물', 0.8091583251953125),
 ('그리움', 0.8079662322998047),
 ('센치', 0.7993037104606628),
 ('외로운', 0.7963045239448547),
 ('혼자', 0.7916884422302246),
 ('우울한', 0.7630760669708252),
 ('슬픈', 0.7467443943023682)]

### **'비'에 감정이 더해지면 결과가 달라진다.**  
'비'만 입력하면 날씨와 관련된 태그(장마, 비올때, 우산, 비오는날)가 주로 나오지만, '외로움'을 추가하면 센치, 우울, 혼자, 새벽감성 등 감정과 관련된 태그들이 주르륵 나온다 !!

In [ ]:
model.w2v_model.wv.most_similar(['비'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('비오는날', 0.8555936813354492),
 ('장마', 0.8403699398040771),
 ('비올때', 0.8138465881347656),
 ('rain', 0.8061612844467163),
 ('우산', 0.7818849086761475),
 ('센치', 0.7765259742736816),
 ('흐린날', 0.768670916557312),
 ('빗소리', 0.7590976357460022),
 ('rainyday', 0.7486249208450317),
 ('비오는', 0.7432389259338379)]

In [ ]:
model.w2v_model.wv.most_similar(['비','외로움'])

[('센치', 0.8622379302978516),
 ('우울', 0.8370097875595093),
 ('쓸쓸한', 0.8238711357116699),
 ('혼자', 0.8085541129112244),
 ('쓸쓸함', 0.7884683609008789),
 ('비오는날', 0.788452684879303),
 ('쓸쓸', 0.7828545570373535),
 ('흐린날', 0.7786180973052979),
 ('새벽감성', 0.7732533812522888),
 ('밤에듣기좋은노래', 0.7567133903503418)]

In [ ]:
model.w2v_model.wv.most_similar(['비오는날','외로움'])

[('비', 0.8661439418792725),
 ('센치', 0.8623786568641663),
 ('우울', 0.8271664381027222),
 ('쓸쓸한', 0.7996366024017334),
 ('혼자', 0.7830052375793457),
 ('쓸쓸', 0.7657393217086792),
 ('흐린날', 0.7645760774612427),
 ('쓸쓸함', 0.7631312608718872),
 ('새벽감성', 0.7469279766082764),
 ('헤어짐', 0.7460042834281921)]

### **태그 합성은 잘하지만, 노래는 잘 찾지 못하는 경향이 있다.**
지코+크러쉬+딘+페노메코+밀릭을 더해서 팬시차일드가 나오지만, 플레이리스트는 다른 가수의 노래가 섞여있다.

In [ ]:
top_items(['지코','크러쉬','딘','페노메코','밀릭'])

> 유사 태그 : ['팬시차일드', 'fanxychild', 'Deanfluenza', 'DEAN', 'ZICO', 'RADMUSEUM', '펀치넬로', 'DPRLIVE', '클럽에스키모', 'Hoody', 'Crush', '하이어뮤직', '오프온오프', '로꼬', 'JayPark', '인디고', '애쉬아일랜드', '쇼미더머니6', '시차', 'ambition']
> 유사곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
470732,"[GN0401, GN0402]",20191205,From Midnight To Sunrise,10361387,[674710],Cloth,[GN0400],[Crush],470732
231100,[GN0401],20191205,From Midnight To Sunrise,10361387,[674710],Wonderlust (Feat. Band Wonderlust),[GN0400],[Crush],231100
225139,"[GN0401, GN0403]",20160617,후미등 (Taillight),2691778,[861257],후미등 (Taillight),[GN0400],[충완],225139
627243,"[GN0205, GN0201]",20160330,BR:evolution,2676262,[724751],지켜만 봐,[GN0200],[소년공화국 (Boys Republic)],627243
108487,"[GN0302, GN0301]",20180521,GOØDevil,10148191,[743761],"춤 (Feat. DEAN, Jinbo)",[GN0300],[Ja Mezz],108487
618180,"[GN0509, GN0501, GN0304, GN0505, GN0301]",20190723,LAKALAKA,10309702,[944663],LAKALAKA,"[GN0500, GN0300]",[LOPE (로페)],618180
601081,"[GN0302, GN0301]",20190909,if.,10327056,[787227],Do Ma Thang,[GN0300],[페노메코 (PENOMECO)],601081
71832,"[GN0302, GN0301]",20180810,L,10193506,[182366],싱가포르 슬링 (Feat. ESBEE) (네모여행 : 싱가포르 편 OST),[GN0300],[이루펀트],71832
520130,"[GN0303, GN0301]",20170904,WALKIN` Vol.2,10093386,[602271],SAY NO X MASTA WU,[GN0300],[PEEJAY],520130
206935,"[GN0501, GN0304, GN0505, GN0301]",20180306,WRITELIGHT,10144693,[944663],Forreal,"[GN0500, GN0300]",[LOPE (로페)],206935


In [ ]:
top_items(['크러쉬'])

> 유사 태그 : ['딘', '밀릭', 'fanxychild', 'DEAN', 'Crush', '팬시차일드', '페노메코', 'Deanfluenza', 'DPRLIVE', 'RADMUSEUM', '펀치넬로', 'ZICO', '클럽에스키모', '박재범', '지코', 'Neosoul', '자이언티', '그레이', 'aomg', 'Hoody']
> 유사곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
231100,[GN0401],20191205,From Midnight To Sunrise,10361387,[674710],Wonderlust (Feat. Band Wonderlust),[GN0400],[Crush],231100
470732,"[GN0401, GN0402]",20191205,From Midnight To Sunrise,10361387,[674710],Cloth,[GN0400],[Crush],470732
67386,"[GN0303, GN0301]",20170916,쇼미더머니 6 Special,10096841,[340669],"bestdriverZ (Feat. Zion.T, DEAN)",[GN0300],[행주],67386
225139,"[GN0401, GN0403]",20160617,후미등 (Taillight),2691778,[861257],후미등 (Taillight),[GN0400],[충완],225139
127406,"[GN0401, GN0403, GN0402]",20161216,On And On,10023711,[735515],Lust (Feat. ELO),[GN0400],[Hoody (후디)],127406
97640,"[GN0401, GN0403, GN0402]",20161216,On And On,10023711,[735515],Like You,[GN0400],[Hoody (후디)],97640
284450,[GN1201],20160115,Malibu (Clean Ver.),10000846,[882497],The Waters (Feat. BJ The Chicago Kid) (Clean V...,[GN1200],[Anderson .Paak],284450
627243,"[GN0205, GN0201]",20160330,BR:evolution,2676262,[724751],지켜만 봐,[GN0200],[소년공화국 (Boys Republic)],627243
416211,"[GN1501, GN0101]",20170415,시카고 타자기 OST Part.2,10055298,[698776],아주 오래된 기억 (Inst.),"[GN1500, GN0100]",[백예린],416211
656987,"[GN0401, GN0402]",20160826,8 Femmes,2707050,[720968],The End (Feat. Paloalto),[GN0400],[ELO],656987


### **어떤 tag는 noise가 되기도 한다.**
* 아이유에 듣고싶은/좋다/좋아 등을 더하게 되면 살짝 엉뚱한 결과가 나온다.
* 큰 의미를 갖지 않는 태그로 예상했지만 걸러줄 필요가 있어보인다.
* 그러나 유사한 태그는 역시 잘 찾는다.

In [ ]:
top_items(['아이유'])

> 유사 태그 : ['IU', '아이유콘서트', '이지금', '아이유노래모음', 'dlwlrma', '유애나', '아이유히트곡', '갓이유', '팔레트', '이지은', 'LovePoem', '아이유셋리스트', '광주', '효리네민박', '밤편지', '이상순', '이효리', '5화', 'JTBC', '메들리']
> 유사곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
630534,[GN0301],20090702,Suga Luv,598564,[299169],Suga Luv (Feat. 아이유) (Inst.),[GN0300],[비즈니즈],630534
166009,"[GN0105, GN0101]",20121221,내 생애 마지막 오디션 - 제3라운드 최후의 듀엣대결 1탄,2170893,"[713967, 713963]","잔소리 (원곡가수 아이유, 임슬옹)",[GN0100],"[이도진, 김연준]",166009
352289,[GN0101],20090618,음악여행 라라라 Live Vol.5,588952,"[197928, 261143, 236880]",All You Need Is Love,[GN0100],"[짙은, 아이유, 란 (RAN)]",352289
582913,[GN0201],20131008,Modern Times,2208448,[261143],기다려,[GN0200],[아이유],582913
452543,[GN0101],20090618,음악여행 라라라 Live Vol.5,588952,[261143],난 사랑을 아직 몰라,[GN0100],[아이유],452543
646278,[GN1701],20131124,Nigerian Beats and Drums,3123703,[712938],The End,[GN1700],[Africana],646278
68324,[GN0201],20091112,IU...IM,718505,[261143],아침 눈물 (Inst.),[GN0200],[아이유],68324
332962,[GN0101],20101117,피아노 가요 연주곡 13,1078052,[465221],"그대네요 (성시경, 아이유)",[GN0100],[뮤직 쿠키],332962
429741,"[GN0601, GN0606]",20191101,Love poem,10346650,[261143],시간의 바깥,[GN0600],[아이유],429741
516905,[GN0201],20131220,Modern Times - Epilogue,2222587,[261143],기다려,[GN0200],[아이유],516905


In [ ]:
top_items(['아이유','듣고싶은'])

> 유사 태그 : ['아이유콘서트', 'dlwlrma', '이지금', '아이유히트곡', '갓이유', 'LovePoem', '광주', '이지은', '아이유노래모음', '아이유셋리스트', '유애나', '팔레트', 'IU', '고음불가', '다양한_장르', '솔로가수', '프리한19', '표절', '가창력종결자', '여자노래방']
> 유사곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
331463,[GN1701],20181207,The Complete Sarah Vaughan On Mercury Vol. 4 -...,5650518,[14186],A Lover&#39;s Concerto,[GN1700],[Sarah Vaughan],331463
618322,[GN0901],20031225,Sleepless In Seoul(31 Greatest High-Quality & ...,33481,[27449],I`m Kissing You,[GN0900],[Des`ree],618322
4996,"[GN1301, GN1303, GN1302]",20021028,The Definitive Collection,2701368,[2106],Sir Duke,[GN1300],[Stevie Wonder],4996
184766,"[GN1501, GN1506, GN1509]",20060328,Ice Age: The Meltdown (Original Motion Picture...,612352,[2279],Foggy Balance,[GN1500],[John Powell],184766
452543,[GN0101],20090618,음악여행 라라라 Live Vol.5,588952,[261143],난 사랑을 아직 몰라,[GN0100],[아이유],452543
303024,[GN0901],20151027,The Greatest Hits,2946442,[2174],Livin` La Vida Loca,[GN0900],[Ricky Martin],303024
447574,[GN2302],20101207,통도사 새벽예불,1231757,[2727],반야심경,[GN2300],[Various Artists],447574
618515,[GN0901],20160812,Die Hit Giganten - Film Hits,2979821,[6018],Can`t Fight The Moonlight,[GN0900],[LeAnn Rimes],618515
331523,"[GN1503, GN1304, GN1501, GN1301]",20090216,B.O.F - De L`autre Cote Du Lit,126382,[28701],Lovin`you,"[GN1500, GN1300]",[Minnie Riperton],331523
567682,[GN0901],20150817,"내 마음을 울린 가슴시린 추억의 음악 (뮤지컬, 영화 OST, 7080 올드 팝 발...",2645637,[2727],Gabriel`s Oboe : Theme From The Mission,[GN0900],[Various Artists],567682


In [ ]:
top_items(['아이유','좋다'])

> 유사 태그 : ['아이유셋리스트', '아이유히트곡', 'LovePoem', '갓이유', '아이유콘서트', '아이유노래모음', 'dlwlrma', '팔레트', '이지은', '이지금', '유애나', '광주', '밤편지', '메들리', '사랑의재개발', 'IU', '18번', '혼코노', '94년생', '코노']
> 유사곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
630534,[GN0301],20090702,Suga Luv,598564,[299169],Suga Luv (Feat. 아이유) (Inst.),[GN0300],[비즈니즈],630534
575265,"[GN0801, GN0802]",20120806,"송창식 (가위, 바위, 보)",2171065,[833],가위 바위 보,[GN0800],[송창식],575265
119850,"[GN1806, GN1801]",20131216,눈 오는 날 골목길 피아노 (Snow),2221597,[727946],군고구마와 군밤,[GN1800],[Littlepiano],119850
332962,[GN0101],20101117,피아노 가요 연주곡 13,1078052,[465221],"그대네요 (성시경, 아이유)",[GN0100],[뮤직 쿠키],332962
522124,[GN2001],20110829,Tassili,2008791,[551807],Takkest Tamidaret,[GN2000],[Tinariwen],522124
357530,"[GN0501, GN0601, GN0503, GN0606, GN0509]",20140728,산사춘 Song 2014,2269681,[756963],산사춘 Song (부제: 당신께 할말이 있어요),"[GN0500, GN0600]",[윤딴딴],357530
210200,"[GN1806, GN1801]",20100730,Landscape,981356,[1410],Coffee For One,[GN1800],[김인주],210200
73321,[GN0101],20100901,피아노 가요 연주곡 9,1007080,[465221],여자라서 (아이유),[GN0100],[뮤직 쿠키],73321
516905,[GN0201],20131220,Modern Times - Epilogue,2222587,[261143],기다려,[GN0200],[아이유],516905
487094,"[GN0501, GN0601, GN0503, GN0606, GN0509]",20141204,넌 나의 우주야,2293601,[21124],넌 나의 우주야 (Feat. Sikstyn),"[GN0500, GN0600]",[오수연],487094


In [ ]:
top_items(['아이유','좋아'])

> 유사 태그 : ['아이유셋리스트', 'LovePoem', '아이유히트곡', '팔레트', '이지은', '아이유콘서트', '이지금', '편함', '갓이유', '우지윤', 'dlwlrma', '풋풋', '94년생', '아이유노래모음', '안지영', '이노래', '유애나', '쏘스윗', '같이듣고싶은노래', '썸탈때']
> 유사곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
201802,"[GN0805, GN1501, GN0801, GN1504]",20190525,최고의 엔딩 OST,10290095,[722722],아무도 모르는 엔딩,"[GN1500, GN0800]",[스텔라장 (Stella Jang)],201802
491901,"[GN0509, GN0105, GN0101, GN0506, GN0501]",20170518,Be There,10063579,[597191],Be There (Radio Ver.),"[GN0500, GN0100]",[CHEEZE (치즈)],491901
635630,"[GN0601, GN0606]",20170925,So Nice (GMF2017 Ver.),10098729,[567072],"So Nice (GMF2017 Ver.) (Feat. 구윤회, 샘김, 오왠, 윤딴딴...",[GN0600],[Grand Mint Band (GMB)],635630
477308,"[GN2502, GN2501, GN0303, GN2504, GN0301]",20191104,Call My Name,10347238,[751059],THURSDAY,"[GN2500, GN0300]",[GOT7 (갓세븐)],477308
317468,[GN1701],20141105,First Sessions,3065632,[1322549],Sugar Town,[GN1700],[Shay Ascarr],317468
146978,"[GN2502, GN0105, GN2501, GN0101, GN2505]",20170921,ANOTHER LIGHT,10098207,[100052],술끊자,"[GN2500, GN0100]",[젝스키스],146978
639160,"[GN0105, GN0101]",20170922,꽃갈피 둘,10096855,[261143],비밀의 화원,[GN0100],[아이유],639160
646833,"[GN0805, GN0501, GN0502, GN0801, GN0509]",20150331,다시 시작,2311660,[717128],달자 (Feat. 림 Of 쏠라티),"[GN0500, GN0800]",[플레이모드],646833
333387,"[GN0805, GN0501, GN0502, GN0801, GN0509]",20170727,ㅎ/,10082935,[699947],5cm (Feat. 김민석 of 멜로망스),"[GN0500, GN0800]",[장희원],333387
645283,"[GN0805, GN0501, GN0502, GN0801, GN0509]",20170412,"조각, 하나",10053934,[746466],아를오오를아,"[GN0500, GN0800]",[최낙타],645283


## **뽑은 태그를 어떻게 넣을 것인가?**
1. 뽑은 그대로 모두 넣자. => `['비', '비오는', '비오는날', '듣고', '듣고싶은', '쓸쓸', '쓸쓸한', '발라드']`

2. 최대로 매칭시켜 넣자. `['비오는날','듣고싶은','쓸쓸한', '발라드']`

In [ ]:
model.w2v_model.wv.most_similar(['비', '비오는', '비오는날', '듣고', '쓸쓸', '쓸쓸한', '발라드'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('센치', 0.8220512866973877),
 ('우울', 0.8118997812271118),
 ('비내리는날음악', 0.8067814707756042),
 ('비올때', 0.7983118295669556),
 ('비내리는날노래', 0.7932559251785278),
 ('비내릴때', 0.7845121026039124),
 ('눈물', 0.7821574211120605),
 ('rainyday', 0.7817319631576538),
 ('헤어짐', 0.7795099020004272),
 ('센치함', 0.7786605358123779)]

In [ ]:
model.w2v_model.wv.most_similar(['비', '비오는', '비오는날', '쓸쓸', '쓸쓸한', '발라드'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('센치', 0.816281795501709),
 ('우울', 0.8121845722198486),
 ('비내리는날음악', 0.7984887361526489),
 ('눈물', 0.7947648763656616),
 ('헤어짐', 0.7944736480712891),
 ('비올때', 0.790336549282074),
 ('rain', 0.7844325304031372),
 ('비내리는날노래', 0.7784298658370972),
 ('비내릴때', 0.7754480838775635),
 ('흐린날', 0.7752397656440735)]

In [ ]:
model.w2v_model.wv.most_similar(['듣고싶은' ,'쓸쓸', '쓸쓸한', '발라드'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('눈물', 0.7937376499176025),
 ('그리움', 0.7879326343536377),
 ('헤어짐', 0.7823807001113892),
 ('외로운', 0.7649414539337158),
 ('슬픈', 0.7643162608146667),
 ('우울할', 0.7586601376533508),
 ('비내리는날음악', 0.7579798698425293),
 ('이별하다', 0.7559695243835449),
 ('이별후', 0.750156044960022),
 ('절절한', 0.7416282296180725)]

In [ ]:
model.w2v_model.wv.most_similar(['여름'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('더위', 0.7738407254219055),
 ('휴가', 0.7679039239883423),
 ('시원한', 0.7590504884719849),
 ('여름노래', 0.7480428218841553),
 ('바캉스', 0.7433536052703857),
 ('해변', 0.7349833846092224),
 ('무더위', 0.7162644863128662),
 ('바다', 0.7126994132995605),
 ('시원', 0.7119068503379822),
 ('청량', 0.6945145130157471)]

In [ ]:
model.w2v_model.wv.most_similar(['여름','듣고싶은'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('여름송', 0.7075084447860718),
 ('614183', 0.6763818264007568),
 ('470608', 0.6753440499305725),
 ('427195', 0.6748813390731812),
 ('327931', 0.6566426753997803),
 ('11200', 0.6534104347229004),
 ('시원한', 0.6529701352119446),
 ('205426', 0.6521424055099487),
 ('더워', 0.6474078297615051),
 ('여름노래모음', 0.6442521214485168)]

# 🌲 **Trie**

In [ ]:
class Node:
    def __init__(self, value):
        self.value = value
        self.children = {}
        self.is_terminal = False

class Trie:
    def __init__(self, items):
        self.head = Node(None)

        print("********* DB 구성중입니다 *********")
        for item in items:
            self.insert(item)
        print("************ 입력 완료 ************")

    def insert(self, query):
        curr_node = self.head

        for q in query:
            if curr_node.children.get(q) is None:
                curr_node.children[q] = Node(q)
            curr_node = curr_node.children[q]
        curr_node.is_terminal = query

    def extract(self, query, biggest_token = True): # two pointer로 만들어야되는구나 ㅠㅠ
        query += '***' # padding
        curr_node = self.head
        prev_node = self.head
        start = 0

        extracted_tags = []

        i = 0 # 현재 위치
        while i < len(query):
            curr_node = curr_node.children.get(query[i])

            if curr_node is None:
                if biggest_token and prev_node.is_terminal:
                    extracted_tags.append(prev_node.is_terminal)
                    # 단어를 발견했으면 start를 jump시키자 => 드라이브 => 드라이브, 이브 와 같은 경우 방지
                    start = i
                else:
                    # 발견하지 못했다면 start를 한칸만 옮기자 => 해변가/로드/라이브 => 해변가, 라이브 와 같은 경우 방지 (jump 시켜버리면 로드트립을 찾고 실패한 후 '라'부터 탐색을 시작해서 드라이브 못찾음.)
                    start += 1
                i = start
                curr_node = self.head
                prev_node = self.head

            else:
                if curr_node.is_terminal:
                    if not biggest_token:
                        extracted_tags.append(curr_node.is_terminal)
                    prev_node = curr_node
                i += 1
 
        return list(set(extracted_tags))

In [ ]:
trie = Trie(model.w2v_model.wv.vocab.keys())

********* DB 구성중입니다 *********
************ 입력 완료 ************


### **가장 많이 매칭되는 태그만을 이용한다.**
찾을 수 있는 모든 태그를 이용하게 되면
* 아이유 => 아이, 아이유 => 잘못된 두 벡터가 더해져 엉뚱한 추천 결과를 제공함
* 듣고싶은 => 듣고, 듣고싶은 => 꼭 필요하지 않은 태그들이 오히려 방해가 될 수 있음.
* 쓸쓸한 => 쓸쓸. 쓸쓸한 => 두 벡터가 유사하게 학습되었으므로 대표 태그 하나만 이용해도 됨. 같은 벡터를 두번 더할 필요는 없다고 판단.

In [ ]:
query = '비오는 날에 듣고 싶은 ㅠㅠ 비오는날 쓸쓸한 발라드'.replace(' ','')

In [ ]:
# 매칭되는 모든 태그 추출

trie.extract(query, False)

['발라드', '비오는', '쓸쓸한', '비오는날', '비', '듣고싶은', '쓸쓸', '듣고']

In [ ]:
# 가장 긴 태그만 추출

trie.extract(query, True)

['듣고싶은', '쓸쓸한', '발라드', '비오는날']

In [ ]:
trie.extract('아이유 노래만 모아모아')

['노래', '아이유']

In [ ]:
trie.extract('또 월요일이라니...출근길 으엘ㅇ륭류 지하철에서 듣고 싶은 노래들'.replace(' ',''))

['듣고싶은노래', '지하철', '월요일', '출근길']

In [ ]:
trie.extract('해변가로 드라이브 떠나고 싶다'.replace(' ',''))

['해변가', '드라이브', '나']

# 📚 **TitleBasedRecommender**

In [ ]:
from itertools import combinations
from collections import defaultdict

class TitleBasedRecommender:
    def __init__(self, w2v_model):
        self.w2v_model = w2v_model
        self.trie = Trie(self.w2v_model.wv.vocab.keys())

    def extract_tags(self, sentence, verbose = True, biggest_token = True):
        sentence = "".join(re.findall('\w', sentence))
        extracted_tags = self.trie.extract(sentence, biggest_token)
        filtered_tags = self.filter_tags(extracted_tags,verbose)

        if verbose:
            print('> Extracted tags :', extracted_tags)
            print("> Filtered tags :",filtered_tags)

        return filtered_tags

    def filter_tags(self, tags, verbose = True):
        if len(tags) <= 2:
            return tags

        pairs = []
        for tag1, tag2 in combinations(tags,2):
            sim = self.w2v_model.wv.similarity(tag1, tag2)
            if sim < 0.1:
                continue
            pairs.append((sim, (tag1, tag2)))

        M,m = max(sim for sim, _ in pairs), min(sim for sim, _ in pairs)
        threshold = m + 0.3*(M-m)

        res = []
        for sim, (tag1, tag2) in pairs:
            if sim < threshold:
                continue
            res.append(tag1)
            res.append(tag2)

        if verbose:
            print('> sims :',pairs)
            print('> threshold :',threshold)

        return list(set(res))
    
    def top_items(self,query_list, verbose = True):
        candidates = self.w2v_model.wv.most_similar(query_list, topn = 1000)
        songs = [item for item,sim in candidates if item.isdigit()]
        tags = [item for item,sim in candidates if not item.isdigit()]

        if verbose:
            print('> 유사 태그 :', tags[:15])
            print('> 유사 곡 :')
            display(song_meta.iloc[songs[:15]])

        return songs, tags

    def title_based_recommend(self, title, verbose = True , biggest_token = True):
        extracted_tags = self.extract_tags(title)
        songs , tags = self.top_items(extracted_tags, verbose)

        if len(songs) < 100:
            print(f'[Warning] recommended songs : {len(songs)}')
        
        # return songs, tags


In [ ]:
rec = TitleBasedRecommender(model.w2v_model)

********* DB 구성중입니다 *********
************ 입력 완료 ************


## **추출한 태그 중에서 유용한 태그만 쓰자**
1. tag가 2개 이하면 그대로 쓴다.
* 레드벨벳+노래 / 아이유+노래 => 부적절한 결과가 나옴. 두개일때도 거를 필요가 있다.
* 듣기좋은, 좋은, 좋다 등 역시 걸러내야할듯.
2. combination을 이용해 모든 pair의 similarity를 계산한다.
* 0.1 미만인 경우 제외한다.
3. 0.3 qunatile 이상인 단어만 선택한다.
4. 선택된 단어가 2개 이하인 경우 return한다. 만약 3개 이상인 경우 자기자신과 유사도가 높은 태그가 2개 이상인 경우만 골라 return한다.

# 👩‍🏫 **제목만 입력해죨!**

##### 🎶 **편집샵 주인장 플레이 리스트 훔쳐왔다 !!!!!**

In [ ]:
songs = rec.title_based_recommend('편집샵 주인장 플레이리스트 훔쳐왔다 !!!!')

> Extracted tags : ['편집샵', '플레이리스트']
> Filtered tags : ['편집샵', '플레이리스트']
> 유사 태그 : ['힙한', '옷가게', '힙스터', '감각적인', '세련된', '편집숍', '트렌디', '패션', '매장음악추천', '힙', '스타일리시', '감각적', '쇼핑', '칠링', '스타일']
> 유사 곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
692170,[GN1101],20140425,"Midnight Lounge Cafe, Vol. 9 - Delicious Loung...",145485,[57354],Breaking (DR Rubberfunk Remix),[GN1100],[Ikon],692170
321751,"[GN2601, GN0509, GN2602, GN0504, GN0501]",20200128,Take Over (feat. Ellie Dixon),10379305,[2764829],Take Over (feat. Ellie Dixon),"[GN0500, GN2600]",[JiNN],321751
295370,"[GN2701, GN2703, GN1104, GN1101]",20180809,Plastic Love,10192612,[2190406],Plastic Love,"[GN2700, GN1100]",[Oddwin],295370
398230,"[GN1104, GN1101]",20150910,Kuaga (Lost Time) Remixes,2639361,[686428],Kuaga (Lost Time) (Leon Lour Remix),[GN1100],[Pierce Fulton],398230
556279,"[GN1102, GN1101]",20171129,Float,10114952,[2639436],A Place,[GN1100],[Fernando],556279
517849,"[GN1109, GN1102, GN1101]",20181109,Dark,10238754,[871813],Hurting (Feat. AlunaGeorge),[GN1100],[SG Lewis],517849
522170,"[GN0908, GN0901]",20190425,FLEXIBLE (Feat. T-Pain) (Remix),10277490,[2636669],FLEXIBLE (Feat. T-Pain) (Remix),[GN0900],[Jordan McGraw],522170
271796,[GN0901],20190523,"Haus Party, Pt. 1",10355844,[1275471],"Nails, Hair, Hips, Heels",[GN0900],[Todrick Hall],271796
578867,[GN0901],20170324,Make Time,10048497,[1073994],Make Time,[GN0900],[Quinn XCII],578867
127990,"[GN0908, GN0901]",20190621,Late Night Feelings,10227376,[45077],Late Night Feelings (feat. Lykke Li),[GN0900],[Mark Ronson],127990


##### 🎶 **스타벅스에서 커피 한 잔 하는 중**

In [ ]:
songs = rec.title_based_recommend('스타벅스에서 커피 한 잔 하는 중이야')

> Extracted tags : ['커피한잔', '스타벅스']
> Filtered tags : ['커피한잔', '스타벅스']
> 유사 태그 : ['차한잔', '홈카페', '모닝음악', '밀크티', '스벅', '디저트', '재즈_pick', '카페뮤직', '카페인', '차가운공기', '쌀쌀한날씨', '카페테라스', '모닝커피', '모닝', '봄밤']
> 유사 곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
230395,[GN1701],20171024,A FINE DAY,10104794,[1272095],A FINE DAY,[GN1700],[Jazz Story],230395
706614,[GN1701],20171107,MISSING YOU,10109177,[1628583],MISSING YOU,[GN1700],[Sweet Note],706614
508184,[GN1701],20170208,Honeysuckle Rose,10036728,[1384357],Honeysuckle Rose,[GN1700],[Morning Post],508184
71548,[GN1701],20170807,Slow Life,10085487,[1628405],Slow Life,[GN1700],[Love Piano],71548
678028,[GN1701],20180404,Hi Spring,10154142,[1814466],Hi Spring,[GN1700],[Smile Again],678028
511775,[GN1701],20171101,Just Friend,10107428,[1705263],Just Friend,[GN1700],[Rainbow],511775
265786,[GN1701],20180314,It`s a Sunshine Day,10147117,[2110336],It`s a Sunshine Day,[GN1700],[Morning Bridge],265786
39351,"[GN1701, GN1707]",20160817,Broken Bossa,2704528,[640632],Valsa Curitibana,[GN1700],[Diego Figueiredo],39351
440141,[GN1701],20180502,Smile of Love,10163112,[2111856],Smile of Love,[GN1700],[Lemon Tree],440141
667940,[GN1701],20170307,Moonlight Garden,10043488,[790828],Moonlight Garden,[GN1700],[Cloudio Trio],667940


##### 🎶 **청량한 남자 아이돌들은 여기로 모이시오 !!!!**

In [ ]:
songs = rec.title_based_recommend('청량한 남자 아이돌들 여기로 모이시오 !!!!')

> sims : [(0.4523165, ('청량한', '남자아이돌')), (0.14991535, ('청량한', '시')), (0.2376937, ('남자아이돌', '시'))]
> threshold : 0.24063569009304048
> Extracted tags : ['청량한', '남자아이돌', '시']
> Filtered tags : ['청량한', '남자아이돌']
> 유사 태그 : ['청량청량', '아이돌노래', '보이그룹', '남돌', '스파클링', '아이돌명곡', '남자아이돌의', '보이그룹노래', '댄스댄스', '신나요', '청량', '아이돌댄스', '예정', '남자아이돌명곡', '인스피릿']
> 유사 곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
540115,"[GN0205, GN0201]",20190911,상상 속의 너,10327423,[2402489],상상 속의 너,[GN0200],[DONGKIZ],540115
359206,"[GN2502, GN0205, GN2501, GN2506, GN0201]",20170823,IDENTITY,10089996,[1066850],말도 안돼,"[GN2500, GN0200]",[VICTON (빅톤)],359206
172366,"[GN2502, GN0205, GN2501, GN2506, GN0201]",20180905,THE BOYZ 1st Single Album [THE SPHERE],10201311,[1816126],L.O.U,"[GN2500, GN0200]",[더보이즈 (THE BOYZ)],172366
14535,"[GN2502, GN0205, GN2501, GN2506, GN0201]",20191021,꿈의 장: MAGIC,10341654,[2632253],Angel Or Devil,"[GN2500, GN0200]",[투모로우바이투게더],14535
668746,"[GN0205, GN0201]",20180704,Golden Child 1st Single Album [Goldenness],10181863,[1907513],LET ME,[GN0200],[골든차일드],668746
299233,"[GN0205, GN0201]",20170828,Golden Child 1st Mini Album (Gol-Cha!),10091533,[1907513],SEA,[GN0200],[골든차일드],299233
11893,[GN0201],20170828,Golden Child 1st Mini Album (Gol-Cha!),10091533,[1907513],네가 너무 좋아,[GN0200],[골든차일드],11893
585036,"[GN2502, GN0205, GN2501, GN2506, GN0201]",20190220,Aside,10253146,[1865968],바람 같은 너 (Feat. 창빈 of Stray Kids),"[GN2500, GN0200]",[윤지성],585036
142519,"[GN2502, GN0205, GN2501, GN2506, GN0201]",20170612,CEREMONY,10070254,[1201990],Lucky,"[GN2500, GN0200]",[펜타곤],142519
39280,"[GN0205, GN0201]",20190717,Cabinet,10307990,"[473980, 2070384]",Cabinet,[GN0200],"[효민, Justatee]",39280


##### 🎶 **왠지 와인을 마셔야할 것 같은, 느낌 충만 음악**

In [ ]:
songs = rec.title_based_recommend('왠지 와인을 마셔야할 것 같은, 느낌 충만 음악')

> sims : [(0.5686929, ('와인', '느낌충만')), (0.2729575, ('와인', '음악')), (0.2774707, ('느낌충만', '음악'))]
> threshold : 0.36167812943458555
> Extracted tags : ['와인', '느낌충만', '음악']
> Filtered tags : ['와인', '느낌충만']
> 유사 태그 : ['wine', 'JAZZY', '내방에서', '소울팝', '와인바', '여유롭게', '관능적', '졸린', '재즈바', 'Bar에서', '커피마시며', '호텔라운지', 'CAFEMUSIC', '가을재즈', '브라스']
> 유사 곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
222284,"[GN1701, GN1706]",20160421,"Mercy, Mercy, Mercy Part.2",2680722,[52104],I Thought About You,[GN1700],[Kuriya Makoto],222284
636129,[GN0901],20101116,...Featuring Norah Jones,1077156,[182154],Love Me,[GN0900],[The Little Willies],636129
559364,[GN0301],20160420,Hip Hop Instrumental Vol. 13 (힙합 비트),2680459,[741754],Groove Love,[GN0300],[비트 팩토리],559364
544055,"[GN1912, GN1909, GN1901]",20051214,SfkuanK!!,392910,[256683],Misirlou `Pulp Fiction`,[GN1900],[SfkuanK!!],544055
226480,[GN1701],20150421,4urlisteningpleasure... (Feat. Marqueal Jordan),3258380,[1140246],All_Day_Everyday (Feat. Marqueal Jordan & Nick...,[GN1700],[DJ I.N.C],226480
602819,"[GN1910, GN1905, GN1912, GN1901]",20170425,In Ya Mellow Tone 13,10057676,[494901],End Of Sorrow,[GN1900],[Robert De Boron],602819
673249,"[GN1701, GN1706]",20170221,Gentaro Takahashi Sings Jazz With Akira Matsuo...,10040193,[1385071],The Rose Tattoo,[GN1700],[Gentaro Takahashi],673249
231579,"[GN1301, GN1302]",20130315,At The Back Of Beyond,2230395,[751559],Thinkin` About You,[GN1300],[Ntjam Rosie],231579
702703,"[GN1701, GN1706]",20170126,Guilty Part.3,10033744,"[1382790, 1382791]",S`Wonderful,[GN1700],"[Mari Kobayashi, Sokonote]",702703
128008,"[GN1701, GN1706]",20171109,ヤスコ·ミ&#12540;ツ·ルイス ライヴ·アット·ミスタ&#12540;·ケリ&#125...,10109519,[1957584],エンブレイサブル&#12539;ユ&#12540; / Embraceable You,[GN1700],[Nakatani Yasuko],128008


##### 🎶 **오늘 하루도 수고한 나에게 위로를**

In [ ]:
songs = rec.title_based_recommend('오늘 하루도 수고한 나에게 위로를')

> sims : [(0.5396966, ('오늘', '수고')), (0.5518309, ('오늘', '나에게')), (0.559791, ('오늘', '하루')), (0.4150514, ('오늘', '위로')), (0.51871645, ('수고', '나에게')), (0.51747596, ('수고', '하루')), (0.5318725, ('수고', '위로')), (0.40885675, ('나에게', '하루')), (0.42172998, ('나에게', '위로')), (0.6350924, ('하루', '위로'))]
> threshold : 0.47672743797302247
> Extracted tags : ['오늘', '수고', '나에게', '하루', '위로']
> Filtered tags : ['오늘', '수고', '나에게', '하루', '위로']
> 유사 태그 : ['지치고', '하루끝', '위로곡', '현실', '괜찮아요', '다잘될거야', '기운내', '취업', '대화', '힘든날', '잘하고있어', '고생했어', '위로가_필요할_때', '바쁜일상', '재충전']
> 유사 곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
585499,"[GN1806, GN1801]",20171018,Autumn Sea,10102960,[944561],"오늘, 수고한 나에게",[GN1800],[달빛바다],585499
251666,"[GN0105, GN0101]",20140124,맘처럼,2228824,[753294],맘처럼,[GN0100],[한길],251666
233123,[],20191016,낮과 밤,10339566,[714975],내일에게,[],[스무살],233123
673518,"[GN1701, GN1703]",20070212,"Fish, Out From Water",344981,[218445],다시 일어나서 걷겠어,[GN1700],[NY물고기],673518
371214,"[GN0601, GN0606]",20151012,DAYBREAK LIVE SUMMER MADNESS 2015 : The Strings,2644220,[229114],빛나는 사람,[GN0600],[데이브레이크 (DAYBREAK)],371214
359260,"[GN1501, GN1504]",20191102,꽃길만 걸어요 OST Part.01,10346675,[484881],다 잘될거야,"[GN1500, GN0200]",[제이세라],359260
560950,[GN1801],20180328,치유의 음악을 찾아 편안히 잠드는 얼후 릴렉싱,10151500,[992622],花は&#21682;く/ Hanawa Saku (꽃은 핀다),[GN1800],[Gan jianmin],560950
227964,"[GN1807, GN1801]",20181031,自律神&#32076;にここちよい音&#27005; ピアノ&#12539;ク&#12540...,10217396,[2400332],TAKUMI,[GN1800],[Yukiko Isomura],227964
82839,[],20200115,비,10375891,[874005],그대에게,"[GN0500, GN0100]",[소래],82839
139091,"[GN0105, GN0101]",20170315,겨울 지나서 봄이 오면,10046023,[1626113],겨울 지나서 봄이 오면,[GN0100],[혜성],139091


##### 🎶 **여름엔 역시 걸그룹이지 ~**

In [ ]:
songs = rec.title_based_recommend('여름엔 역시 걸그룹이지 ~')

> sims : [(0.26517707, ('걸그룹', '역시')), (0.6278947, ('걸그룹', '여름엔')), (0.20900832, ('역시', '여름엔'))]
> threshold : 0.33467423021793363
> Extracted tags : ['걸그룹', '역시', '여름엔']
> Filtered tags : ['걸그룹', '여름엔']
> 유사 태그 : ['여돌', '걸그룹댄스', '걸그룹댄스곡', '걸그룹명곡', '여자댄스곡', '위키미키', '여자아이돌', 'ITZY', '프듀48', '귀여워', '청순', '있지', '열대야에듣는음악', '아이돌의숨은명곡돌', '파랑']
> 유사 곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
381812,"[GN2503, GN0205, GN2501, GN2506, GN0201]",20190604,SPECIAL ALBUM [For the Summer],10293846,[943959],눈부셔,"[GN2500, GN0200]",[우주소녀],381812
657278,"[GN0205, GN0201]",20190819,We Got The Power,10318521,[1757346],We Got The Power,[GN0200],[소녀주의보],657278
543222,[],20200203,reminiscence,10383682,[2631992],SALUTE,[],[EVERGLOW (에버글로우)],543222
491731,"[GN2503, GN0205, GN2501, GN2506, GN0201]",20190805,OH MY GIRL SUMMER PACKAGE [FALL IN LOVE],10314743,[857994],다섯 번째 계절 (SSFWL),"[GN2500, GN0200]",[오마이걸],491731
444210,"[GN2503, GN0205, GN2501, GN2506, GN0201]",20180820,[+ +],10191989,[1229429],Perfect Love,"[GN2500, GN0200]",[이달의 소녀],444210
498950,"[GN0205, GN0201]",20190704,유학소녀 (UHSN),10303087,[2737554],팝시클 (POPSICLE),[GN0200],[유학소녀 (UHSN)],498950
659255,"[GN0205, GN0201]",20180612,포도포도해,10174885,[2018506],포도포도해 (Grapes),[GN0200],[버스터즈],659255
161982,[],20200422,에이프릴(APRIL) 7th Mini Album &#39;Da Capo&#39;,10421256,[882818],인형,[],[에이프릴 (APRIL)],161982
685425,"[GN2503, GN0205, GN2501, GN2506, GN0201]",20191223,‘The ReVe Festival’ Finale,10368053,[780066],음파음파 (Umpah Umpah),"[GN2500, GN0200]",[Red Velvet (레드벨벳)],685425
406394,"[GN2503, GN0205, GN2501, GN2506, GN0201]",20190723,밤의 공원(THE PARK IN THE NIGHT) part three,10310108,[2295762],Total Eclipse (Black Out),"[GN2500, GN0200]",[공원소녀 (Girls in the Park)],406394


##### 🎶 **없던 첫사랑도 떠오르는 달달한 사랑 노래**

In [ ]:
songs = rec.title_based_recommend('없던 첫사랑도 떠오르는 달달한 사랑 노래')

> sims : [(0.7103138, ('첫사랑', '사랑노래')), (0.5708393, ('첫사랑', '달달한')), (0.62922883, ('사랑노래', '달달한'))]
> threshold : 0.6126816391944885
> Extracted tags : ['첫사랑', '사랑노래', '달달한']
> Filtered tags : ['첫사랑', '사랑노래', '달달한']
> 유사 태그 : ['연애', '썸', '애인', '달달', '커플', '고백', '달달함', '오래된연인', '두근두근', '연애세포', '스윗', '연인', '로맨스', '데이트', '콩닥콩닥']
> 유사 곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
545850,"[GN0105, GN0101]",20190328,좋아서 그래 좋아서 그래,10266534,[439620],좋아서 그래 좋아서 그래(feat.포롱(Polong)),[GN0100],[페퍼민트 (peppermint)],545850
602825,"[GN0501, GN0502, GN0801, GN0509]",20170313,누나야,10045179,[1383862],누나야,"[GN0500, GN0800]",[아란],602825
653763,"[GN0105, GN0101]",20190628,연습해온 말,10300595,[2400673],"연습해온 말(with김주성,박지혜)",[GN0100],[멜로틱],653763
292015,"[GN0501, GN0101, GN0506, GN0509]",20171112,드루와,10110558,[674794],오늘밤은 그대와 잠들고 싶어,"[GN0500, GN0100]",[소심한 오빠들],292015
175855,"[GN0805, GN0501, GN0502, GN0801, GN0509]",20180812,핑크핑크해,10193292,[967002],핑크핑크해,"[GN0500, GN0800]",[이리원],175855
424280,[GN0101],20170929,내 여자라는게,10100266,[1382184],내 여자라는게,[GN0100],[인환],424280
225184,"[GN0105, GN0101]",20190929,Wedding,10333316,[576751],Wedding,[GN0100],[김원주],225184
330151,"[GN0501, GN0101, GN0506, GN0509]",20180718,"봄, 그대에게",10185626,[1628269],"봄, 그대에게","[GN0500, GN0100]",[복잡쟁이],330151
415085,"[GN0805, GN0501, GN0502, GN0801, GN0509]",20180810,단짠단짠,10193224,[1201498],얘가 이렇게 예뻤나,"[GN0500, GN0800]",[훈스 (HOONS)],415085
599056,"[GN0501, GN0601, GN0503, GN0606, GN0509]",20180417,nervous,10158725,[890106],조마조마해,"[GN0500, GN0600]",[초빈 (Chobin)],599056


##### 🎶 **하이틴 영화 속 주인공인 척 ...**

In [ ]:
songs = rec.title_based_recommend('하이틴 영화 속 주인공인 척 .....')

> sims : [(0.36799684, ('하이틴', '영화')), (0.70923793, ('하이틴', '주인공')), (0.55485016, ('영화', '주인공'))]
> threshold : 0.4703691691160202
> Extracted tags : ['하이틴', '영화', '주인공']
> Filtered tags : ['하이틴', '주인공', '영화']
> 유사 태그 : ['내가사랑했던모든남자들에게2', '토이스토리', '외국영화', '인어공주', '알라딘', '라푼젤', '해외영화', '영화수록곡', '라이온킹', 'Aladdin', '주디', 'disney', '영화사운드트랙', '스파이더맨', '디즈니OST']
> 유사 곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
459934,"[GN1503, GN1014, GN1501, GN1003, GN1001]",20200207,To All The Boys: P.S. I Still Love You (Music ...,10385842,[710272],Way Back In,"[GN1500, GN1000]",[AgesandAges],459934
360178,[GN2001],20160916,Kitsune Maison Compilation 17: World Wild Issue,3948700,[1944544],Psychic Reader,[GN2000],[Bad Bad Hats],360178
325350,"[GN1014, GN1008, GN1001]",20181012,Singles,10211762,[2111566],Tal Uno,[GN1000],[Barrie],325350
216829,"[GN1503, GN1501]",20200207,You Should Be Dancing (From The Netflix Film “...,10386218,[989837],You Should Be Dancing,"[GN1500, GN0900]",[The New Respects],216829
399539,"[GN1503, GN1501, GN0901]",20170310,Beauty and the Beast (Original Motion Picture ...,10309137,[2275],How Does A Moment Last Forever (From &#34;Beau...,"[GN1500, GN0900]",[Celine Dion],399539
379566,"[GN0908, GN0901]",20190920,Mood Swing,10329810,[1816560],I Can’t Believe,[GN0900],[Cyn],379566
329305,[GN1701],20140310,Then He Kissed Me,4243465,[29391],Then He Kissed Me,[GN1700],[The Crystals],329305
484587,[GN0901],20200207,As I&#39;ll Ever Be (From The Netflix Film “To...,10385348,[2764382],As I&#39;ll Ever Be (From The Netflix Film “To...,[GN0900],[Chaz Cardigan],484587
444103,"[GN1509, GN2207, GN1003, GN1001, GN1501, GN1506]",20181115,Ralph Breaks the Internet (Original Motion Pic...,10236888,[635412],Zero (From the Original Motion Picture &#34;Ra...,"[GN1500, GN2200, GN1000]",[Imagine Dragons],444103
529694,"[GN1509, GN1506, GN1501]",20190621,Toy Story 4 (Banda Sonora Original en Castellano),10298448,[30594],You&#39;ve Got a Friend in Me (From &#34;Toy S...,[GN1500],[Randy Newman],529694


##### 🎶 **틀어놓기만 해도 해변가 드라이브 중 !**

In [ ]:
songs = rec.title_based_recommend('틀어놓기만 해도 해변가 드라이브 중 ~')

> Extracted tags : ['해변가', '드라이브']
> Filtered tags : ['해변가', '드라이브']
> 유사 태그 : ['수영장', '시원한곡', '여름휴가철', '여름나기', '썸머쏭', '여름여행', '시원해지는', '신나는', '트로피칼', '바캉스', '여름엔', '덩실덩실', '청량하게', '시원시원', '풀파티']
> 유사 곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
223701,[GN0901],19000101,Ai Se Eu Te Pego,2950903,"[685130, 232722]",Ai Se Eu Te Pego (If I Get Ya) (Worldwide Remix),[GN0900],"[Michel Telo, Pitbull]",223701
476171,"[GN2701, GN2702, GN2703, GN1103]",20190809,Colors,10316391,[591892],Colors,"[GN2700, GN1100]",[The Knocks],476171
241649,[GN2601],20181226,Fired,10236416,[2296258],Fired,[GN2600],[DRINC],241649
308748,"[GN1104, GN1103, GN1101]",20190628,Fantasy,10301178,[560126],Fantasy (Feat. Ruckazoid),[GN1100],[Breakbot],308748
549540,"[GN2701, GN2702, GN2703, GN1103]",20190426,Find Yourself (Remixes),10278634,"[786510, 717437]",Find Yourself (Ashworth Remix),"[GN2700, GN1100]","[Great Good Fine OK, Before You Exit]",549540
247347,"[GN1102, GN1101]",20160909,Money Maker,2711183,[748312],Money Maker (Feat. Lunchmoney Lewis & Aston Me...,[GN1100],[Throttle],247347
555453,[GN1101],20141124,Bondax & Friends - The Mix Album,2962393,[709675],Dusk Funk,[GN1100],[Bondax],555453
375261,[GN0201],20190118,All Right,10243263,[1229336],All Right,[GN0200],[C-Korn],375261
655715,"[GN1105, GN1109, GN1101]",20190705,Kind of Blue,10303321,[743235],Kind of Blue,[GN1100],[Hellberg],655715
511999,[GN2601],20161207,Controlless,10021076,[1229336],Controlless,[GN2600],[C-Korn],511999


##### 🎶 **퇴근 후 맥주 한잔이 땡기는 날**

In [ ]:
songs = rec.title_based_recommend('퇴근 후 맥주 한잔이 땡기는 날')

> Extracted tags : ['퇴근후', '맥주한잔']
> Filtered tags : ['퇴근후', '맥주한잔']
> 유사 태그 : ['저녁길', 'refresh', '휴식할', '한강산책', '해질무렵', '무드있고', '준비중', '다섯', '새벽_감성', '스프링', '맥주펍', '저녁산책', '날씨좋은날', '나른한노래', '초저녁']
> 유사 곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
282854,"[GN0908, GN0901]",20180928,"summer,",10252905,[1758307],selfish,[GN0900],[Jeremy Zucker],282854
157395,[GN1301],20191206,Good Daze (Feat. Shanin Blake),10361443,[2025152],Good Daze (Feat. Shanin Blake),[GN1300],[Jazzinuf],157395
184592,"[GN0908, GN0901]",20190524,Figure It Out - EP,10289148,[748237],Running Cold,[GN0900],[Jai Waetford],184592
598608,"[GN0401, GN0402]",20191223,MIX #4,10367725,[889407],Freeze,[GN0400],[Ban:jax (밴젝스)],598608
526736,"[GN0908, GN0901]",20180119,vertigo,10140188,[176178],love; not wrong (brave),[GN0900],[EDEN],526736
542206,"[GN1014, GN0901, GN0902, GN1001]",20190401,Dizzy (Mahogany Sessions),10268037,[1628573],Dizzy (Mahogany Sessions),"[GN0900, GN1000]",[Tim Atlas],542206
23726,"[GN2601, GN0509, GN2603, GN0504, GN0501]",20200218,LUCID,10389615,[1183362],Eyes (feat. 가은 (GA EUN)),"[GN0500, GN2600]",[JIDA (지다)],23726
175295,"[GN0908, GN0901]",20181029,"Betty, Pt. 1",10216712,[753590],Ignore Me,[GN0900],[Betty Who],175295
17438,"[GN0908, GN0901]",20190426,Hurts 2B Human,10252959,[10385],My Attic,[GN0900],[Pink],17438
59696,"[GN1105, GN1109, GN1101]",20190711,More Than Friends,10305505,[967934],More Than Friends,[GN1100],[Mokita],59696


##### 🎶 **벌써 월요일이라고???? 출근길 힐링송**

In [ ]:
songs = rec.title_based_recommend('벌써 월요일이라고???? 출근길 힐링송')

> sims : [(0.3660692, ('힐링송', '월요일')), (0.33681756, ('힐링송', '출근길')), (0.6838055, ('월요일', '출근길'))]
> threshold : 0.4409139513969421
> Extracted tags : ['힐링송', '월요일', '출근길']
> Filtered tags : ['월요일', '출근길']
> 유사 태그 : ['아침', '월요병', '시작', '모닝콜', '출근', '상쾌한', '지하철', '피곤', '퇴근길', '모닝', '기분업', '일할때', '주말', '버스', '작업']
> 유사 곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
36240,[GN1701],20171020,High Drive,10103932,[1703876],High Drive,[GN1700],[Mood Swing],36240
621907,[GN1801],20180302,In Love,10143564,[2059927],In Love,[GN1800],[Lewisia (레위시아)],621907
451866,[GN1801],20180319,I Am You,10148992,[965966],I Am You,[GN1800],[별 헤는 밤],451866
112932,[GN1801],20180103,거기 그대로..,10124178,[2059927],거기 그대로..,[GN1800],[Lewisia (레위시아)],112932
458543,"[GN1806, GN1801]",20151207,Piano Nostalgie,2655036,[699949],언제나 몇번이라도,[GN1800],[허정현],458543
601472,"[GN0805, GN0501, GN0502, GN0801, GN0509]",20120411,어느 날 갑자기 어른이 되었다,2109449,[581352],커피 내리는 시간,"[GN0500, GN0800]",[마더컨트리],601472
209847,[GN1801],20180614,시간을 넘어서,10175878,[750758],시간을 넘어서,[GN1800],[Ueno Miki],209847
377906,[GN1801],20171113,별을 쏘다,10110599,[906244],별을 쏘다,[GN1800],[행복을 너에게],377906
41562,[GN1801],20171024,잠시 쉬어가기,10104962,[927876],잠시 쉬어가기,[GN1800],[시엘 (Ciel)],41562
595974,"[GN1806, GN1801]",20171016,Stars Falling From The Sky,10102274,[699949],Stars Falling From The Sky,[GN1800],[허정현],595974


##### 🎶 **내적 그루브 유발하는 끈적끈적 노래들**

In [ ]:
songs = rec.title_based_recommend('내적 그루브 유발하는 끈적끈적 노래들')

> sims : [(0.273342, ('노래들', '끈적끈적')), (0.5389901, ('노래들', '유발')), (0.3281918, ('노래들', '내적그루브')), (0.65944624, ('끈적끈적', '유발')), (0.5695376, ('끈적끈적', '내적그루브')), (0.7030405, ('유발', '내적그루브'))]
> threshold : 0.4022515535354614
> Extracted tags : ['노래들', '끈적끈적', '유발', '내적그루브']
> Filtered tags : ['노래들', '내적그루브', '끈적끈적', '유발']
> 유사 태그 : ['urban', '그루브팝', 'Tinashe', '매장음악카페', 'Feel', '컨템포러리RNB', 'NAO', 'Neosoul', '옷가게매장', 'BlackMusic', '그루브한노래', '야한음악', 'Alternativernb', 'RKelly', '소울팝']
> 유사 곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
701979,"[GN1304, GN1301, GN1302]",20140526,Me. I Am Mariah…The Elusive Chanteuse (Deluxe),2257805,[2274],Thirsty (Album Version (Explicit)),[GN1300],[Mariah Carey],701979
635493,"[GN1301, GN1302]",20161111,Nightride,10014299,[752524],Company,[GN1300],[Tinashe],635493
319653,"[GN1301, GN1302]",20170428,Strength Of A Woman,10051195,[28403],Telling The Truth (Feat. KAYTRANADA & BADBADNO...,[GN1300],[Mary J. Blige],319653
190851,[GN1301],20150518,All This Love,223747,[103756],All This Love,[GN1300],[DeBarge],190851
521266,"[GN1102, GN1101]",20150826,Re-Flowed,2336360,[743067],Do What I Do (Feat. Omar) (Peejay Remix),[GN1100],[Positive Flow],521266
323090,"[GN1301, GN1302]",20170217,Trust,10039430,[881460],Naked,[GN1300],[Will Heard],323090
456666,[GN1201],20160812,Bad Boy 20th Anniversary Box Set Edition,2703645,[100728],Only You (Feat. The Notorious B.I.G. & Mase) (...,[GN1200],[One Twelve],456666
258636,"[GN1301, GN1302]",20170609,COLOURS 2,10069335,[727602],Low Battery,[GN1300],[PARTYNEXTDOOR],258636
641854,"[GN1301, GN1302]",20161118,Her,10017590,[467800],Me & You,[GN1300],[Angel],641854
194434,"[GN1301, GN1302]",20170324,Tremaine The Album,10039874,[172573],1x1,[GN1300],[Trey Songz],194434


##### 🎶 **마음이 몽글몽글해지는 팝송**

In [ ]:
songs = rec.title_based_recommend('마음이 몽글몽글해지는 팝송')

> sims : [(0.20762262, ('몽글몽글', '팝송')), (0.28995144, ('몽글몽글', '마음이')), (0.43078524, ('팝송', '마음이'))]
> threshold : 0.27457140386104584
> Extracted tags : ['몽글몽글', '팝송', '마음이']
> Filtered tags : ['몽글몽글', '팝송', '마음이']
> 유사 태그 : ['두근거리는', '때껄룩', '언제들어도', 'popmusic', '잔잔한팝송', '나른한노래', '새벽갬성', '33분', '베스트11', '알렉벤자민', 'refresh', '밤길', 'I_love__this', '원필추천', '귀차니즘']
> 유사 곡 :


,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
566478,[GN0901],20200227,Cape God,10394947,[1640569],Love Me Wrong (feat. Troye Sivan),[GN1100],[Allie X],566478
7039,[GN0901],20190129,Sine Qua Non,10246713,[1134534],Chariot,[GN0900],[Jacob Lee],7039
130933,[GN0901],20200221,~how i&#39;m feeling~,10337847,[1956543],Billy,[GN0900],[Lauv],130933
588166,[GN0901],20200117,Manic,10376518,[861336],Without Me,[GN0900],[Halsey],588166
624371,[GN0901],20200221,~how i&#39;m feeling~,10337847,[1956543],Canada (feat. Alessia Cara),[GN0900],[Lauv],624371
326061,"[GN0908, GN0901]",20191008,untitled,10337128,[176178],untitled,[GN0900],[EDEN],326061
566564,[GN0901],20200221,~how i&#39;m feeling~,10337847,[1956543],Believed,[GN0900],[Lauv],566564
147411,"[GN0908, GN0901]",20190913,Free Time,10316352,[1438336],Unsaid,[GN0900],[Ruel],147411
23706,[GN0901],20200221,~how i&#39;m feeling~,10337847,[1956543],For Now,[GN0900],[Lauv],23706
514610,[GN1001],20200106,Newgame+,10371646,"[2761952, 2761954]",Don&#39;t Cha Mind...,[GN0900],"[vito., Jess Xaviera]",514610
